In [1]:
import os
import h5py
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from modis_utils.misc import cache_data, restore_data

In [3]:
data = restore_data('cache/boundary_vectors_ALL.h5')

In [4]:
train_boundary_vectors = data[0]
val_boundary_vectors = data[1]
test_boundary_vectors = data[2]

In [5]:
n_points = train_boundary_vectors.shape[1]
n_points

1024

In [6]:
train_boundary_vectors.shape, val_boundary_vectors.shape, test_boundary_vectors.shape

((438, 1024, 2), (138, 1024, 2), (138, 1024, 2))

In [7]:
def transform(data, scaler, flatten=True):
    old_shape = data.shape
    data = data.reshape(old_shape[0], -1)
    data = scaler.transform(data.astype(np.float))
    if not flatten:
        return data.reshape(old_shape)
    return data

In [8]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_boundary_vectors.reshape(train_boundary_vectors.shape[0], -1))
train_boundary_vectors_scale = transform(train_boundary_vectors, scaler, flatten=True)
val_boundary_vectors_scale = transform(val_boundary_vectors, scaler, flatten=True)
test_boundary_vectors_scale = transform(test_boundary_vectors, scaler, flatten=True)

In [9]:
# normalize the dataset
train_boundary_vectors_scale_1 = transform(train_boundary_vectors, scaler, flatten=False)
val_boundary_vectors_scale_1 = transform(val_boundary_vectors, scaler, flatten=False)
test_boundary_vectors_scale_1 = transform(test_boundary_vectors, scaler, flatten=False)

In [10]:
train_boundary_vectors_scale_1.shape

(438, 1024, 2)

In [11]:
def create_dataset(boundary_vectors_scale, timesteps):
    data_X = []
    data_Y = []
    for i in range(len(boundary_vectors_scale) - timesteps):
        data_x = boundary_vectors_scale[i:(i+timesteps)]
        data_y = boundary_vectors_scale[i + timesteps]
        data_X.append(data_x)
        data_Y.append(data_y)
    return np.asarray(data_X), np.asarray(data_Y)

In [12]:
timesteps = 50
train_X, train_Y = create_dataset(train_boundary_vectors_scale, timesteps)
val_X, val_Y = create_dataset(np.concatenate(
    [train_boundary_vectors_scale[-timesteps:], val_boundary_vectors_scale]),
                              timesteps)
test_X, test_Y = create_dataset(np.concatenate(
    [val_boundary_vectors_scale[-timesteps:], test_boundary_vectors_scale]),
                                timesteps)

In [13]:
timesteps = 50
train_X_1, train_Y_1 = create_dataset(train_boundary_vectors_scale_1, timesteps)
val_X_1, val_Y_1 = create_dataset(np.concatenate(
    [train_boundary_vectors_scale_1[-timesteps:], val_boundary_vectors_scale_1]),
                              timesteps)
test_X_1, test_Y_1 = create_dataset(np.concatenate(
    [val_boundary_vectors_scale_1[-timesteps:], test_boundary_vectors_scale_1]),
                                timesteps)

In [14]:
train_X.shape, train_Y.shape, val_X.shape, val_Y.shape, test_X.shape, test_Y.shape

((388, 50, 2048),
 (388, 2048),
 (138, 50, 2048),
 (138, 2048),
 (138, 50, 2048),
 (138, 2048))

In [15]:
train_X_1.shape, train_Y_1.shape, val_X_1.shape, val_Y_1.shape, test_X_1.shape, test_Y_1.shape

((388, 50, 1024, 2),
 (388, 1024, 2),
 (138, 50, 1024, 2),
 (138, 1024, 2),
 (138, 50, 1024, 2),
 (138, 1024, 2))

In [16]:
def create_graph_matrix(n_points_on_boundary):
    def calc_arc_distance(a, b, n):
        diff = np.abs(a-b)
        if diff > n//2:
            diff = n - diff
        return diff
    
    n = n_points_on_boundary
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = calc_arc_distance(i, j, n)
    return mat.astype(np.float32)

def create_graph_matrix_1(n_points_on_boundary):
    def calc_arc_distance(a, b, n):
        diff = np.abs(a-b)
        if diff > n//2:
            diff = n - diff
        return diff
    
    n = n_points_on_boundary
    mat = np.zeros((2*n, 2*n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = calc_arc_distance(i, j, n)
    mat[n:2*n, n:2*n] = mat[:n, :n]
    for i in range(n):
        for j in range(n, 2*n):
            mat[i,j] = mat[i, j - n]
    mat[n:2*n, :n] = mat[:n, n:2*n]
    return mat.astype(np.float32)

In [17]:
mat = create_graph_matrix(n_points)
mat.shape

(1024, 1024)

In [18]:
class DotDict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)

In [19]:
import sys
import random
import datetime

import torch
import torch.nn as nn
import torch.optim as optim

sys.path.append('../../GRNN/grnn/')
from model import GRNN

In [20]:
opt = DotDict()
opt.nNode = n_points
opt.batchSize = 1
opt.dimHidden = 32
opt.dimFeature = 2
opt.truncate = 50
opt.nIter = 2
opt.cuda = False
opt.lr = 0.01

In [21]:
data_np = np.expand_dims(train_boundary_vectors_scale_1, axis=0)

In [22]:
data_np.shape

(1, 438, 1024, 2)

In [23]:
data = torch.from_numpy(data_np)
A = torch.from_numpy(mat[np.newaxis, :, :]).double()

In [24]:
net = GRNN(opt)
net.double()
print(net)

GRNN(
  (propogator): Propogator(
    (gruCell_0): gruCell(
      (resetGate): Sequential(
        (0): Linear(in_features=34, out_features=32, bias=True)
        (1): Sigmoid()
      )
      (updateGate): Sequential(
        (0): Linear(in_features=34, out_features=32, bias=True)
        (1): Sigmoid()
      )
      (transform): Sequential(
        (0): Linear(in_features=34, out_features=32, bias=True)
        (1): Tanh()
      )
      (output): Linear(in_features=32, out_features=2, bias=True)
    )
    (gruCell_1): gruCell(
      (resetGate): Sequential(
        (0): Linear(in_features=34, out_features=32, bias=True)
        (1): Sigmoid()
      )
      (updateGate): Sequential(
        (0): Linear(in_features=34, out_features=32, bias=True)
        (1): Sigmoid()
      )
      (transform): Sequential(
        (0): Linear(in_features=34, out_features=32, bias=True)
        (1): Tanh()
      )
      (output): Linear(in_features=32, out_features=2, bias=True)
    )
    (gruCell_2): g

In [25]:
criterion = nn.MSELoss()

In [26]:
if opt.cuda:
    net.cuda()
    criterion.cuda()
    data = data.cuda()
    A = A.cuda()

In [27]:
optimizer = optim.Adam(net.parameters(), lr=opt.lr)

In [28]:
hState = torch.randn(opt.batchSize, opt.dimHidden, opt.nNode).double()
yLastPred = 0

In [29]:
def getTime(begin, end):
    timeDelta = end - begin
    return '%d h %d m %d.%ds' % (timeDelta.seconds // 3600, (timeDelta.seconds%3600) // 60, timeDelta.seconds % 60, timeDelta.microseconds)

timStart = datetime.datetime.now()

In [40]:
point_plot_idx = 0
feature_plot_idx = 1

In [38]:
x = data[:, :, 0, 1]

In [39]:
x

tensor([[ 0.4000,  0.4000,  0.4500,  0.4500,  0.5500,  0.6000,  0.7500,
          0.6000,  0.5000,  0.5500,  0.5500,  0.5500,  0.5500,  0.5000,
          0.4500,  0.4500,  0.5500,  0.5000,  0.5500,  0.4500,  0.5500,
          0.5000,  0.4500,  0.4500,  0.4500,  0.4500,  0.3000,  0.2500,
          0.2500,  0.2500,  0.3000,  0.2500,  0.2000,  0.2500,  0.2000,
          0.2000,  0.1000,  0.0500,  0.0500,  0.0500,  0.0000,  0.0000,
          0.0500,  0.1000,  0.1000,  0.2000,  0.2500,  0.4000,  0.4000,
          0.2500,  0.2500,  0.4500,  0.5500,  0.5500,  0.5000,  0.5000,
          0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5500,  0.5000,
          0.5000,  0.5500,  0.5000,  0.5500,  0.4000,  0.4000,  0.3000,
          0.3500,  0.3000,  0.2500,  0.2000,  0.3000,  0.4000,  0.4000,
          0.0500,  0.1000,  0.0500,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0500,
          0.1000,  0.1000,  0.2500,  0.2000,  0.4000,  0.4500,  

In [ ]:
for t in range(data.size(1) - opt.truncate):
    x = data[:, t:(t + opt.truncate), :, :]
    y = data[:, (t + 1):(t + opt.truncate + 1), :, :]

    for i in range(opt.nIter):
        process = '[Log] %d propogation, %d epoch. ' % (t + 1, i + 1)
        timStamp = datetime.datetime.now()
        prediction, hNew = net(x, hState, A)
        print(process + 'Forward used: %s.' % getTime(timStamp, datetime.datetime.now()))
        hState = hState.data

        loss = criterion(prediction, y)
        optimizer.zero_grad()
        timStamp = datetime.datetime.now()
        loss.backward()
        print(process + 'Backward used: %s.' % getTime(timStamp, datetime.datetime.now()))
        optimizer.step()

    _, hState = net.propogator(x[:, 0, :, :], hState, A)
    hState = hState.data

    if t == 0:
        #plt.plot([v for v in range(opt.truncate)], 
        #         x[:, :, point_plot_idx, feature_plot_idx].data.numpy().flatten(), 'r-')
        plt.plot([v + 1 for v in range(opt.truncate)], 
                 prediction[:, :, point_plot_idx, feature_plot_idx].data.numpy().flatten(), 'b-')
    else:
        #plt.plot([t + opt.truncate - 2, t + opt.truncate - 1], 
        #         x[:, -2:, point_plot_idx, feature_plot_idx].data.numpy().flatten(), 'r-')
        plt.plot([t + opt.truncate - 1, t + opt.truncate],
                 [yLastPred, prediction[0, -1, point_plot_idx, feature_plot_idx]], 'b-')
    plt.draw()
    plt.pause(0.5)
    yLastPred = prediction[0, -1, point_plot_idx, feature_plot_idx]

plt.ioff()
plt.show()